# Nest

Prepares data for the Nest page 

In [1]:
import os
from events import (
    load_event_data, filter_public_events, filter_by_project, prepare, save_files, count_wards
)

Set output directory

In [2]:
OUT_DIR = '../../../../docs/metrics/events/nest/_data/events'

Read in event data and filter by **My LEEDS 2023** project.

The criteria are:

* project type is 'Produce (Signature)'
* event_type contains 'Public Event -'

It's then filtered by project_name is '10 - Melt'

In [4]:
data = (
  load_event_data()
    .pipe(filter_by_project, '10 - Melt')
  )
data

,actual_audience_size_number_of_participants_in_person,actual_audience_size_number_of_participants_online,airtable_id,end_date,event_name,event_type,event_unique_identifier,number_of_booked_participants,postcode_(from_venue),project_name,project_type,season,start_date,ticket_tailor_id,ward_(from_venue)
499,NaN,NaN,242,2023-04-06,R&D Workshop - Young People/Tech,['CLE - Youth Voice session'],10 - Melt: R&D Workshop - Young People/Tech,NaN,NaN,10 - Melt,Produce (Signature),['Season 3 - Dreaming'],2023-04-03,NaN,NaN
509,NaN,67.0,2032,2023-04-10,NEST: Audition Video Applications,"['CLE - Skills and Talent session', 'CLE - Par...",10 - Melt: NEST: Audition Video Applications,NaN,NaN,10 - Melt,Produce (Signature),NaN,2023-04-10,NaN,NaN
639,47.0,NaN,244,2023-04-30,NEST: Acting Intake Auditions,['CLE - Participant / Volunteer thank you even...,10 - Melt: NEST: Acting Intake Auditions,NaN,NaN,10 - Melt,Produce (Signature),['Season 2 - Playing'],2023-04-29,NaN,NaN
783,20.0,NaN,245,2023-06-03,NEST: Acting Intake May Course,['CLE - Participant / Volunteer thank you even...,10 - Melt: NEST: Acting Intake May Course,NaN,['LS2 7UP'],10 - Melt,Produce (Signature),['Season 2 - Playing'],2023-05-30,NaN,['Little London and Woodhouse']
822,50.0,NaN,1629,2023-06-11,NEST: Protagonist Auditions,['CLE - Skills and Talent session'],10 - Melt: NEST: Protagonist Auditions,NaN,['LS2 7UP'],10 - Melt,Produce (Signature),['Season 2 - Playing'],2023-06-10,NaN,['Little London and Woodhouse']
969,11.0,NaN,1776,2023-07-06,NEST: Theatre Show Trip,"['CLE - Skills and Talent session', 'CLE - Par...",10 - Melt: NEST: Theatre Show Trip,NaN,['LS2 7UP'],10 - Melt,Produce (Signature),['Season 2 - Playing'],2023-07-06,NaN,['Little London and Woodhouse']
987,7.0,NaN,243,2023-07-12,NEST: Protagonist Auditions 2nd Round,"['CLE - Skills and Talent session', 'CLE - Par...",10 - Melt: NEST: Protagonist Auditions 2nd Round,NaN,['LS10 1NE'],10 - Melt,Produce (Signature),['Season 1 - Awakening'],2023-07-12,NaN,['Hunslet and Riverside']
1009,29.0,NaN,625,2023-08-04,NEST: Acting Intake Course Compass Collective,['CLE - Participant / Volunteer thank you even...,10 - Melt: NEST: Acting Intake Course Compass ...,NaN,['LS2 7BS'],10 - Melt,Produce (Signature),['Season 2 - Playing'],2023-07-24,NaN,['Little London and Woodhouse']
1010,15.0,NaN,2031,2023-08-04,NEST: Acting Intake July Course 2,['CLE - Participant / Volunteer thank you even...,10 - Melt: NEST: Acting Intake July Course 2,NaN,['LS2 7UP'],10 - Melt,Produce (Signature),NaN,2023-07-24,NaN,['Little London and Woodhouse']
1023,12.0,NaN,397,2023-08-04,NEST: Acting Intake July Course 1,['CLE - Participant / Volunteer thank you even...,10 - Melt: NEST: Acting Intake July Course 1,NaN,['LS2 7UP'],10 - Melt,Produce (Signature),['Season 2 - Playing'],2023-07-27,NaN,['Little London and Woodhouse']


Run the processing and save the files

In [24]:
data = data.pipe(filter_public_events).pipe(prepare).pipe(save_files, OUT_DIR)